In [29]:
# PACOTES

import pandas as pd
import requests
import os
import json
import re
from dotenv import load_dotenv

In [30]:
# PARÂMETROS

load_dotenv(override=True)
access_token = os.getenv("access_token")
headers = {'X-GitHub-Api-Version': '2022-11-28','Authorizarion': 'Baerer' + access_token}
owner = 'amzn'
url = 'https://api.github.com'
url_formatted = f'{url}/users/{owner}/repos'

In [31]:
# EXTRAÇÃO

response = requests.get(url_formatted, headers=headers)

if response.status_code:
    print('success')
else:
    print('error')

success


In [32]:
# PAGINAÇÃO

if response.status_code == 200:
    link_header = response.headers.get('Link')
    if link_header:
        links = link_header.split(',')
        last_url = None
        for link in links:
            if 'rel="last"' in link:
                last_url = link.split(';')[0].strip('<>')
                break

if last_url:
    match = re.search(r'page=(\d+)', last_url)
    if match:
        last_page = match.group(1)
        last_page = int(last_page)

print(last_page)

6


In [33]:
# EXTRAÇÃO GERAL

repos_list = []

for page_num in range(1,last_page+1):
    try:
        url_page = f'{url_formatted}?page={page_num}'
        response = requests.get(url_page, headers=headers)
        repos_list.append(response.json())
    except:
        repos_list.append(None)

In [34]:
# EXTRAÇÃO DAS LINGUAGENS

repos_languages = []

for page in repos_list:
    for repo in page:
        repos_languages.append(repo['language'])

repos_languages[:5]

[None, 'Jupyter Notebook', 'Smarty', 'JavaScript', None]